# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [16]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

df = pd.read_csv('/datasets/data.csv')
df.info()
df.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**



После изучения общей информации можно увидеть первые неточности в данных:
1. Имеются пропущенные значения NaN в столбцах days_employed и total_income.
2. Данные столбцы имеют имеют тип данных float.
3. Некоторые данные days_employed имеют отрицательное значение. Так же непонятны большие значения в этом столбце, так как по элементарным подсчетам получается, что количество стажа в годах превышвет возраст клиента. Возможно данные выражены не в днях, а в другой величине.
4. Столбец education имеет явные дубликаты.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
df.isna().sum()

При подсчете пропусков видим, что столбцы days_employed и total_income имеют равное значение 2174, это почти 9% от общих значений. Для начала необходимо определить к какому типу занятости относится клиенты с пропущенными значениями стажа и дохода, и понять есть ли там закономерность, т.к. в первых пятидесяти строках пропущенные значения выпадают только на госслужащих и пенсионеров. 
Задача состоит в том, чтобы определить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, поэтому пропущенные значения имеют второстепенный характер и их можно заполнить средними значениями из каждой категории.

In [ ]:
df['days_employed'] = df['days_employed'].apply(abs)
df['total_income'] = df['total_income'].apply(abs)

Но, для корректного подсчета среднего значения необходимо привести столбцы days_employed и total_income к целым числам, и понять откуда такие большие, а так же отрицательные значения и так же преобразовать их. Отрицательные значения преобразуем по модулю.

In [ ]:
df['days_employed'].hist()

df.loc[df['days_employed'] > 16000, 'days_employed'] = df.loc[df['days_employed'] > 16000, 'days_employed'] / 24

Для посчета среднего значения, предполагаем, что большие значения в столбце days_employed выражены не в днях, а в часах. 
Для преобразования днных к необходимому формату выявляем максимальный и минимальный порог с помощью графика. 
Для этого через hist() строим графики. Наглядно получаем необходимый порог.
После преобразуем значения выше порога из часов в сутки.

Далее заполняем пропущенные значения.
Для этого определяем тип занятости клиентов по которым нет данных days_employed и total_income. 
После группировки видим, что пропуски есть по 5 видам занятости: госслужащий, компаньон, пенсионер, предприниматель, сотрудник, высчитываем среднее значение стажа и дохода по этим видам и заполняем пробелы.

In [ ]:
for income in df['income_type'].unique(): 
    median_days = df.loc[df['income_type'] == income, 'days_employed'].median() 
    median_income = df.loc[df['income_type'] == income,'total_income'].median()
    
    df.loc[(df['days_employed'].isna()) & (df['income_type'] == income),'days_employed'] = median_days 
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income),'total_income'] = median_income

    
# Просле делаем проверку через isna() на отсутствие пропусков.
df.isna().sum()

Далее проверяем все столбцы на уникальное значение, в столбце dob_years есть значение 0, т.е. у 101 клиента не указан возраст, так же заменяем нули медианой по типу занятости.
В столбце gender у одного клиента не указан пол. Данное значение оставим неизменным, т.к. оно не несет в себе ключевой информации.

В столбце children есть 47 отрицательных значений -1 и 76 значений с количеством 20. После применения фильтра стало понятно, что это скорее всего опечатка,т.к. многим клиентам нет даже 40 лет.

In [17]:
for income in df['income_type'].unique():
    median_year = df.loc[df['income_type'] == income, 'dob_years'].median()
    df.loc[(df['dob_years'] == 0) & (df['income_type'] == income),'dob_years'] = median_year
    
for children in df['children'].unique():
    df.loc[(df['children'] == 20) & (df['children'] == children), 'children'] = 2
    df.loc[(df['children'] == -1) & (df['children'] == children), 'children'] = 0
    
# Преобразуем -1 в 0, а 20 в 2.

**Вывод**

-1 мог образоваться из-за неккоректной выгрузки или из-за объединения нескольких систем, где данные отображаются в разном формате, т.е -1 предполагает "нет детей" - 0.
20 могло стать послествием человеческого факора, опечатка. 2 и 0 очень близко находятся на клавиатуре и при быстром зполнении образовалась неточность. Эти данные не могут быть корректными, т.к. большинству клиентов нет даже 40 лет для такого количества детей.

### Замена типа данных

In [3]:
#df.info()

df['days_employed'] = df['days_employed'].astype('int')
df['dob_years'] = df['dob_years'].astype('int')

**Вывод**

days_employed и dob_years пренадлежит к типу данных float, который необходимо пребразовать в int, для этого используем метод astype(). 
total_income трогать не стоит, т.к. нам неизвестно в какой валюте заполнен доход каждого клиента.

### Обработка дубликатов

In [4]:
df.duplicated().sum()

df['education'] = df['education'].str.lower()
df = df.drop_duplicates().reset_index(drop=True)
#df.info()

**Вывод**

При проверке через медот duplicated(), было выявленно 53 строки. После приведения строк в колонке education к нижнему регистру, количество дубликаов увеличилось до 71. После их удаления была произведена проверка через info(). 
Возникновение дубликатов может быть спровоцировано сбоем в выгрузе или например если информация была выгружена из нескольких систем.

### Лемматизация

In [5]:
# Через функцию unique() получаем уникальные значения purpose и склеваем их в один список с помощью join().

purpose_list = df['purpose'].unique()
purpose_list = m.lemmatize(' '.join(purpose_list))

Counter(purpose_list)

# Подсчитываем частоту появления слов. 
# Выделяем основные категории: жилье, автомобиль, образование, недвижимость, свадьба, операция, строительство.
# Далее прописываем функцию соответствующую данной логике.

def categorize(purpose_list):
    purpose_lemmas = m.lemmatize(purpose_list)
    if 'недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas:
        return 'недвижимость'
    if 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    if 'образование' in purpose_lemmas:
        return 'образование'
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'
    return 'прочее'

df['category'] = df['purpose'].apply(categorize)

#display(df.loc[:, ['purpose', 'category']])
#df.head()

**Вывод**

C помощью метода apply() создаем новый столбец category.
Через display выводим результат.

### Категоризация данных

In [33]:
df[['income_type', 'total_income']].drop_duplicates()
df[['children']].drop_duplicates()

# С помощью drop_duplicates() находим списки для объединения в категории.

def children_quantity(children):
    if children == 0:
        return 'нет детей'
    return 'есть дети'
df['children_quantity'] = df['children'].apply(children_quantity)


def type_level(income_type):
    if income_type == 'сотрудник' or 'компаньон' or 'госслужащий' or 'предприниматель':
        return 'работает'
    if income_type == 'безработный' or 'в декрете' or 'студент':
        return 'не работает'
    return 'пенсионер'
df['type_level'] = df['income_type'].apply(type_level)


def income_level(total_income):
    if total_income <= 100000:
        return 'низкий уровень дохода'
    if total_income <= 180000:
        return 'средний уровень дохода'
    return 'высокий уровень дохода'
df['income_level'] = df['total_income'].apply(income_level)

In [34]:
#код ревьюера
df['income_level'].value_counts()

средний уровень дохода    10639
высокий уровень дохода     6423
низкий уровень дохода      4463
Name: income_level, dtype: int64

**Вывод**

В категоризации выделили основные категории и привели их в более сжатый вид. На основе этих категорий создали новые столбцы в таблице.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [30]:
children_debt = df.pivot_table(index = 'children_quantity', columns = 'debt', values = 'children', aggfunc = 'count')
children_debt.columns = ['no_debt', 'debt']
children_debt['% невозврата'] = (children_debt['debt']/(children_debt['debt'] + children_debt['no_debt']))*100
display(children_debt)



children_quantity_debt = df.pivot_table(index = 'children', columns = 'debt', values = 'children_quantity', aggfunc = 'count')
children_quantity_debt.columns = ['no_debt', 'debt']
children_quantity_debt['% невозврата'] = (children_quantity_debt['debt']/(children_quantity_debt['debt'] + children_quantity_debt['no_debt']))*100
children_quantity_debt


,no_debt,debt,% невозврата
children_quantity,,,
есть дети,6652,677,9.237277
нет детей,13132,1064,7.495069


,no_debt,debt,% невозврата
children,,,
0,13132.0,1064.0,7.495069
1,4374.0,444.0,9.215442
2,1929.0,202.0,9.479118
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


**Вывод**

Семьи с детьми возвращаю кредит в срок реже чем семьи без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [31]:
family_status_debt = df.pivot_table(index = ['family_status'], columns = 'debt', values = 'family_status_id', aggfunc = 'count')
family_status_debt.columns = ['no_debt', 'debt']
family_status_debt['% невозврата'] = (family_status_debt['debt']/(family_status_debt['debt'] + family_status_debt['no_debt']))*100
family_status_debt

,no_debt,debt,% невозврата
family_status,,,
Не женат / не замужем,2539,274,9.740491
в разводе,1110,85,7.112971
вдовец / вдова,897,63,6.562500
гражданский брак,3789,388,9.288963
женат / замужем,11449,931,7.520194


**Вывод**

Женатые, овдовевшие или разведеные клиенты отдают кредит в срок чаще, чем не женатые клиенты.
Скорее вего это связано с тем, что семейное положение обязует к формированию общего бюджета, где заранее просчитываются все необходимые расходы. Так же у женатых людей бюджет чаще всего формируется из двух и более источников дохода, т.е эта категория наименее уязвима по отношению политическим, экономическим кризисам и т.д.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
income_debt = df.pivot_table(index = 'income_level', columns = 'debt', values = 'total_income', aggfunc = 'count')
income_debt.columns = ['no_debt', 'debt']
income_debt['% невозврата'] = (income_debt['debt']/(income_debt['debt'] + income_debt['no_debt']))*100
income_debt

,no_debt,debt,% невозврата
income_level,,,
высокий уровень дохода,5945,478,7.442005
низкий уровень дохода,4109,354,7.931884
средний уровень дохода,9730,909,8.544036


**Вывод**

Клиенты с низким и высоким уровнем дохода, отдают кредит в срок чаще. Разрыв небольшой чуть больше 1%. Скорее всего это связано с тем, что на средний уровень дохода приходится большая часть клиентов. Просрочки в оплате долгов могут происходить из-за жизненных ситуаций и человеческого фактора.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
category_debt = df.pivot_table(index = 'category', columns = 'debt', values = 'total_income', aggfunc = 'count')
category_debt.columns = ['no_debt', 'debt']
category_debt['%'] = (category_debt['debt']/(category_debt['debt'] + category_debt['no_debt']))*100
category_debt

,no_debt,debt,%
category,,,
автомобиль,3903,403,9.359034
недвижимость,10029,782,7.233373
образование,3643,370,9.220035
свадьба,2138,186,8.003442


**Вывод**

Клиенты взявшие кредит на приобретение недвижимости отдают долг в срок чаще, чем кленты с другими целями. Скорее всего это связано с первычными потребностями человека, статься без дома намного страшнее чем, например, потерять автомобиль. Конечно качество жизни ухудшиться, но это не так критично, как остаться без дома.

# Вывод

Семейное положение имеет влияние на факт погашения кредита в срок. От количества детей практически ничего не зависит, а вот наличие детей так же имеет влияние на кредитную историю клиента.